# Relocation Analysis
In this project, I am looking to develop a data analysis package that can assess similarity between the current neighborhood that I am living in against neighborhoods within Bangkok, Thailand.

## Aims of the Project
The idea came around as I am in the process of relocating from United Kingdom to Thailand as an expat, and would like to evaluate neighborhoods within Bangkok and find similar neighborhood that shares some similarity to my current neighborhood - Edgbaston, Birmingham, UK. 

## The Problems
I have not been to Thailand for a long time and have very limited knowledge about Bangkok and where is a good place to live. This is a very challenging issue for me as a person who is soon to be living in a foreign city. Any additional knowledge that could help me pick a neighborhood to live in would be fantastic! Nobody wants to relocate half the world away only to live in an unsuitable area far away from everything.

In addition to finding similar neighborhoods based on types of venues obtained from Foursquare, I want to take a deeper look at these venues and narrow down the list further based on types of venues I want to have closeby. The following are criterias that I am looking for:
<ol>
<li>Gym: within 2000m</li>
<li>Coffee Shop: within 1000m</li>
<li>Convenience Store/Grocery Store/Supermarket: within 2000m</li>
</ol>

## The Solutions
In this project, I will look to obtain venues data within the radius of my current neighborhood - Edgbaston, Birmingham, UK and compare it against similar venues data of all neighborhoods in Bangkok, Thailand. These venues data will be obtained from Foursquare, and the neighborhoods coordinates for Bangkok will be scraped from Wikipedia via the following url.
https://en.wikipedia.org/wiki/List_of_districts_of_Bangkok

Once a list of similar neighborhood was obtained, I will filter out results to look for a neighborhood that contains Gym, Coffee Shop, and Grocery Store within a set distance to finalise the search.

-----------------------------------------------------------------

# Section 1: Obtaining Data
In this section, I am looking to scrape Bangkok's neighborhoods coordinates from Wikipedia (https://en.wikipedia.org/wiki/List_of_districts_of_Bangkok) and find Point of Interests (venues) data from Foursquare. Similar process will also be carried out against my current neighborhood as well.

In [1]:
#import relevant packages
import pandas as pd
from geopy.geocoders import Nominatim

In [2]:
#scrape coordinates of Bangkok's neighborhood from Wikipedia
bangkok_url = 'https://en.wikipedia.org/wiki/List_of_districts_of_Bangkok'

bangkok_data = pd.read_html(bangkok_url, match='District')

In [3]:
#cleaned up data by removing unneeded columns and rename appropriately
bangkok_data = bangkok_data[0].drop(columns=['MapNr','Thai','Popu-lation','No. ofSubdis-trictsKhwaeng'], axis=1)
bangkok_data.columns = ['District','Post Code','Latitude','Longitude']

print("Shape of Table: {} rows and {} columns.".format(bangkok_data.shape[0] ,bangkok_data.shape[1]))
bangkok_data.head()

Shape of Table: 50 rows and 4 columns.


,District,Post Code,Latitude,Longitude
0,Bang Bon,10150,13.659200,100.399100
1,Bang Kapi,10240,13.765833,100.647778
2,Bang Khae,10160,13.696111,100.409444
3,Bang Khen,10220,13.873889,100.596389
4,Bang Kho Laem,10120,13.693333,100.502500


In [4]:
#use Nominatim to get coordinates of my current neighborhood
geolocator = Nominatim(user_agent='relocation')
location = geolocator.geocode('B5 7SU Birmingham UK')
print(location.address)

print((location.latitude, location.longitude))

Birmingham, West Midlands Combined Authority, West Midlands, England, B5 7SU, United Kingdom
(52.454127889769914, -1.9051928110854555)


In [5]:
#create DataFrame to store similar data of my current neighborhood
bham_data = pd.DataFrame({'District':['Edgbaston'],'Post Code':['B5 7SU'],'Latitude': [location.latitude],'Longitude': [location.longitude]})
bham_data

,District,Post Code,Latitude,Longitude
0,Edgbaston,B5 7SU,52.454128,-1.905193


# Section 2: Get Venues Data from Foursquare
The section below gets venues data from Foursquare through its API and perform data cleaning/transforming to shape the data into a workable format.

In [6]:
#importing relevant packages
import requests
from pandas.io.json import json_normalize
import numpy as np

In [7]:
#creating url for Foursquare API to get venues
CLIENT_ID = 'H4XUBXBTKFVT5Q1ZTIBLIBMPPTXTUOH0LHK4T0TRKPRHKOQW' # your Foursquare ID
CLIENT_SECRET = 'LEDR45BDQPMK0JLIQBUVQQDFYA5U350WO0MFDZG2AN2IZSWK' # your Foursquare Secret
VERSION = '20200701' # Foursquare API version

#limiting number of venues to 100 and radius of 1000m
radius = 3000
LIMIT = 100

In [8]:
#create a function to generate get url to obtain venues data from Foursquare
def getNearbyVenues(names, latitudes, longitudes, rad):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            rad, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
#Run the getNearbyVenues function to get venues data for Bangkok and store in dataframe
bangkok_poi = getNearbyVenues(names=bangkok_data['District'],
                                   latitudes=bangkok_data['Latitude'],
                                   longitudes=bangkok_data['Longitude'],
                                   rad = radius
                                  )
print(bangkok_poi.shape)
bangkok_poi.head()

(4774, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bang Bon,13.6592,100.3991,ขาหมูบางหว้า,13.657136,100.395230,Thai Restaurant
1,Bang Bon,13.6592,100.3991,ร้านต้นไม้ ริมถนนกาญจนาภิเษก,13.654098,100.405054,Garden Center
2,Bang Bon,13.6592,100.3991,TPD Bowling,13.663977,100.408965,Bowling Alley
3,Bang Bon,13.6592,100.3991,KFC,13.670449,100.405502,Fast Food Restaurant
4,Bang Bon,13.6592,100.3991,Burger King (เบอร์เกอร์คิง),13.670830,100.405089,Fast Food Restaurant


In [10]:
#Run the getNearbyVenues function to get venues data for my current neighborhood and store in dataframe

bham_poi = getNearbyVenues(names=bham_data['District'],
                                   latitudes=bham_data['Latitude'],
                                   longitudes=bham_data['Longitude'],
                                   rad = radius
                                  )
print(bham_poi.shape)
bham_poi.head()

(100, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Edgbaston,52.454128,-1.905193,Cannon Hill Park,52.452584,-1.903403,Park
1,Edgbaston,52.454128,-1.905193,mac birmingham,52.452857,-1.903515,Performing Arts Venue
2,Edgbaston,52.454128,-1.905193,Edgbaston Cricket Ground,52.455610,-1.903759,Cricket Ground
3,Edgbaston,52.454128,-1.905193,Bridges Cafe Bar & Arena Bar,52.452894,-1.903544,Café
4,Edgbaston,52.454128,-1.905193,Damascena,52.446345,-1.888391,Coffee Shop


In [11]:
#joining to two POI tables for simplicity
poi = pd.concat([bangkok_poi, bham_poi])
poi.reset_index(drop=True, inplace = True)

print(poi.shape)
poi.head()

(4874, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bang Bon,13.6592,100.3991,ขาหมูบางหว้า,13.657136,100.395230,Thai Restaurant
1,Bang Bon,13.6592,100.3991,ร้านต้นไม้ ริมถนนกาญจนาภิเษก,13.654098,100.405054,Garden Center
2,Bang Bon,13.6592,100.3991,TPD Bowling,13.663977,100.408965,Bowling Alley
3,Bang Bon,13.6592,100.3991,KFC,13.670449,100.405502,Fast Food Restaurant
4,Bang Bon,13.6592,100.3991,Burger King (เบอร์เกอร์คิง),13.670830,100.405089,Fast Food Restaurant


In [12]:
# one hot encoding
onehot = pd.get_dummies(poi[['Venue Category']], 
                                prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['District'] = poi['District'] 

# move neighborhood column to the first column
n_col = onehot.columns.get_loc('District')
if n_col == 0:
    fixed_columns = list(onehot.columns[:])
else:
    fixed_columns = [onehot.columns[n_col]] + list(onehot.columns[0:n_col]) + list(onehot.columns[n_col+1:])

onehot = onehot[fixed_columns]

print(onehot.shape)
onehot.head()

(4874, 262)


,District,Airport,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,...,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Bang Bon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bang Bon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bang Bon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bang Bon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bang Bon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#Group data into each neighborhood by mean value
poi_grouped = onehot.groupby('District').mean().reset_index()
poi_grouped.head()

,District,Airport,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,...,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Bang Bon,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,...,0.01,0.01,0.01,0.00,0.00,0.0,0.00,0.0,0.0,0.0
1,Bang Kapi,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.02,0.01,0.01,0.00,0.00,0.0,0.00,0.0,0.0,0.0
2,Bang Khae,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.00,0.01,0.01,0.00,0.00,0.0,0.01,0.0,0.0,0.0
3,Bang Khen,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.03,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0
4,Bang Kho Laem,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.01,0.00,0.00,0.01,0.01,0.0,0.00,0.0,0.0,0.0


In [14]:
#a function to get the top most common venues for each neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [15]:
#set number of top venues to be used
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
poi_sorted = pd.DataFrame(columns=columns)
poi_sorted['District'] = poi_grouped['District']

for ind in np.arange(poi_grouped.shape[0]):
    poi_sorted.iloc[ind, 1:] = return_most_common_venues(
        poi_grouped.iloc[ind, :], num_top_venues)

poi_sorted.head()

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bang Bon,Convenience Store,Coffee Shop,Noodle House,Fast Food Restaurant,Asian Restaurant,Thai Restaurant,Flea Market,Badminton Court,Hotpot Restaurant,Market
1,Bang Kapi,Noodle House,Coffee Shop,Thai Restaurant,Japanese Restaurant,Dessert Shop,Som Tum Restaurant,Fast Food Restaurant,Steakhouse,Clothing Store,Soccer Field
2,Bang Khae,Noodle House,Coffee Shop,Fast Food Restaurant,Convenience Store,Thai Restaurant,Dessert Shop,Asian Restaurant,Café,Shopping Mall,Department Store
3,Bang Khen,Coffee Shop,Noodle House,Convenience Store,Hotpot Restaurant,Asian Restaurant,Thai Restaurant,Som Tum Restaurant,Golf Course,Bookstore,Vietnamese Restaurant
4,Bang Kho Laem,Noodle House,Thai Restaurant,Coffee Shop,Hotel,Asian Restaurant,Pub,Chinese Restaurant,BBQ Joint,Bar,Massage Studio


# Section 3: k-Means Clustering
In this section, I am going to utilise the k-Means Clustering to segregate neighborhoods in to different labels. This should allow me to explore similar neighborhoods using the resulted labels.

I will set the number of clusters to 5 to begin with.

In [16]:
from sklearn.cluster import KMeans

In [17]:
#set number of k-means cluster
kclusters = 5

poi_clustered = poi_grouped.drop('District', 1)

#Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(poi_clustered)

kmeans.labels_[0:51]

array([1, 4, 2, 2, 2, 2, 1, 4, 3, 4, 2, 2, 4, 2, 4, 0, 1, 4, 3, 2, 2, 4,
       3, 0, 2, 1, 2, 1, 1, 1, 3, 1, 0, 2, 0, 0, 1, 4, 3, 1, 3, 4, 3, 4,
       2, 4, 2, 1, 2, 0, 0])

In [18]:
#append clustering labels to data
bangkok_bham_merged = pd.concat([bangkok_data,bham_data]).reset_index(drop=True)

poi_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge poi_grouped with neighborhood data to add latitude/longitude for each neighborhood
bangkok_bham_merged = bangkok_bham_merged.join(poi_sorted.set_index('District'), on='District')

bangkok_bham_merged.head()

,District,Post Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bang Bon,10150,13.659200,100.399100,1,Convenience Store,Coffee Shop,Noodle House,Fast Food Restaurant,Asian Restaurant,Thai Restaurant,Flea Market,Badminton Court,Hotpot Restaurant,Market
1,Bang Kapi,10240,13.765833,100.647778,4,Noodle House,Coffee Shop,Thai Restaurant,Japanese Restaurant,Dessert Shop,Som Tum Restaurant,Fast Food Restaurant,Steakhouse,Clothing Store,Soccer Field
2,Bang Khae,10160,13.696111,100.409444,2,Noodle House,Coffee Shop,Fast Food Restaurant,Convenience Store,Thai Restaurant,Dessert Shop,Asian Restaurant,Café,Shopping Mall,Department Store
3,Bang Khen,10220,13.873889,100.596389,2,Coffee Shop,Noodle House,Convenience Store,Hotpot Restaurant,Asian Restaurant,Thai Restaurant,Som Tum Restaurant,Golf Course,Bookstore,Vietnamese Restaurant
4,Bang Kho Laem,10120,13.693333,100.502500,2,Noodle House,Thai Restaurant,Coffee Shop,Hotel,Asian Restaurant,Pub,Chinese Restaurant,BBQ Joint,Bar,Massage Studio


# Section 4: Mapping Neighborhoods
In this section, I will map the neighborhoods in Bangkok to display markers on each neighborhood and use color scheme to show similar neighborhoods. I will also produce a table showing neighborhoods that have the same labels as 'Edgbaston', my current neighborhood as well.

In [19]:
#!pip install webcolors
import webcolors as wc
#!pip install folium
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

In [20]:
def get_approx_color(hex_color):
    orig = wc.hex_to_rgb(hex_color)
    similarity = {}
    for hex_code, color_name in wc.css3_hex_to_names.items():
        approx = wc.hex_to_rgb(hex_code)
        similarity[color_name] = sum(np.subtract(orig, approx) ** 2)
    return min(similarity, key=similarity.get)

def get_color_name(hex_color):
    try:
        return wc.hex_to_name(hex_color)
    except ValueError:
        return get_approx_color(hex_color)

In [28]:
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#printing marker's color and cluster label for neighborhoods that are similar to my current neighborhood
edgbaston_label = bangkok_bham_merged.loc[bangkok_bham_merged['District']=='Edgbaston', 'Cluster Labels'].values[0]
print("The marker in \033[1m\033[4m{}\033[0m are neighborhoods that share similarity to my current neighborhood, \
which have Cluster Label \033[1m\033[4m{}\033[0m".format(get_color_name(rainbow[edgbaston_label-1]), edgbaston_label) )

# add markers to the map
map_bangkok = folium.Map(location=[13.7563, 100.5018], zoom_start=11)
markers_colors = []
for lat, lon, poi_, cluster in zip(bangkok_bham_merged['Latitude'], bangkok_bham_merged['Longitude'], 
                                  bangkok_bham_merged['District'], bangkok_bham_merged['Cluster Labels']):
    #if loop was used to check for NaN clusters and set color to black
    if np.isnan(cluster):
        cluster = 'No Data'
        colour = 'black'
    else:
        colour = rainbow[int(cluster)-1]
    label = folium.Popup(str(poi_) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colour,
        fill=True,
        fill_color=colour,
        fill_opacity=0.7).add_to(map_bangkok)
       
map_bangkok

The marker in aquamarine are neighborhoods that share similarity to my current neighborhood, which have Cluster Label 3


In [22]:
#Neighborhood with the same cluster label as my current neighborhood
bangkok_bham_merged.loc[bangkok_bham_merged['Cluster Labels'] == (edgbaston_label),
                     bangkok_bham_merged.columns[[0]+[4] + list(range(5,
                        bangkok_bham_merged.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Bang Rak,3,Hotel,Coffee Shop,Bar,Spa,Thai Restaurant,Restaurant,Italian Restaurant,Asian Restaurant,Chinese Restaurant,Café
21,Khlong San,3,Noodle House,Hotel,Seafood Restaurant,Bar,Coffee Shop,Hotel Bar,Asian Restaurant,Chinese Restaurant,Café,Spa
29,Pathum Wan,3,Hotel,Coffee Shop,Noodle House,Dessert Shop,Shopping Mall,Gym / Fitness Center,Movie Theater,Bakery,Seafood Restaurant,Department Store
37,Ratchathewi,3,Hotel,Coffee Shop,Bakery,Shopping Mall,Dessert Shop,Hostel,Department Store,Clothing Store,Café,Movie Theater
39,Samphanthawong,3,Hotel,Noodle House,Seafood Restaurant,Coffee Shop,Bar,Thai Restaurant,Asian Restaurant,Dessert Shop,Chinese Restaurant,Spa
41,Sathon,3,Hotel,Thai Restaurant,Noodle House,Coffee Shop,Asian Restaurant,Bar,Som Tum Restaurant,Spa,Café,Bistro
50,Edgbaston,3,Coffee Shop,Pub,Indian Restaurant,Café,Bar,Restaurant,Hotel,Park,Moroccan Restaurant,Middle Eastern Restaurant


# Section 5: A Deeper Dive into Neighborhood
In addition to finding similar neighborhoods based on types of venues obtained from Foursquare, I want to take a deeper look at these venues and narrow down the list further based on types of venues I want to have closeby. The following are criterias that I am looking for:
<ol>
<li>Gym: within 2000m</li>
<li>Coffee Shop: within 1000m</li>
<li>Convenience Store/Grocery Store/Supermarket: within 2000m</li>
</ol>

In [23]:
#import relevant package
!pip install geopy
import geopy.distance

In [24]:
#Get a series of similar neighborhoods
similar_neigh = bangkok_bham_merged.loc[(bangkok_bham_merged['Cluster Labels'] == (edgbaston_label)) & 
                                        (bangkok_bham_merged['District'] !='Edgbaston'), bangkok_bham_merged.columns[0]]
print(similar_neigh)

#from poi table, get entries that belong to District in similar_neigh
poi_similar_neigh = poi.loc[poi['District'].isin(similar_neigh)].reset_index(drop=True)
poi_similar_neigh.head()

8           Bang Rak
21        Khlong San
29        Pathum Wan
37       Ratchathewi
39    Samphanthawong
41            Sathon
Name: District, dtype: object


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bang Rak,13.730833,100.524167,Rod Dee Det (รสดีเด็ด),13.734807,100.524340,Noodle House
1,Bang Rak,13.730833,100.524167,Amara Bangkok,13.728673,100.527564,Hotel
2,Bang Rak,13.730833,100.524167,iSanook Residence,13.732560,100.521723,Hotel
3,Bang Rak,13.730833,100.524167,สุกี้สามย่าน,13.734000,100.527565,Thai Restaurant
4,Bang Rak,13.730833,100.524167,Jok Sam Yan (โจ๊กสามย่าน),13.734714,100.526080,Breakfast Spot


In [25]:
#define a function to calculate distance between two coordinates in km
def distancer(row):
    coords_1 = (row['District Latitude'], row['District Longitude'])
    coords_2 = (row['Venue Latitude'], row['Venue Longitude'])
    return geopy.distance.distance(coords_1, coords_2).km

#added a column to store calculated distance between District and Venue
poi_similar_neigh['Distance(km)'] = poi_similar_neigh.apply(distancer, axis=1)
poi_similar_neigh.head()

,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Distance(km)
0,Bang Rak,13.730833,100.524167,Rod Dee Det (รสดีเด็ด),13.734807,100.524340,Noodle House,0.440069
1,Bang Rak,13.730833,100.524167,Amara Bangkok,13.728673,100.527564,Hotel,0.438295
2,Bang Rak,13.730833,100.524167,iSanook Residence,13.732560,100.521723,Hotel,0.326184
3,Bang Rak,13.730833,100.524167,สุกี้สามย่าน,13.734000,100.527565,Thai Restaurant,0.507782
4,Bang Rak,13.730833,100.524167,Jok Sam Yan (โจ๊กสามย่าน),13.734714,100.526080,Breakfast Spot,0.476646


In [40]:
#define required venue categories
interested_cat = ('Gym / Fitness Center','Coffee Shop','Convenience Store','Grocery Store','Supermarket')

#filter DataFrame based on stated criteria in description above
summary = poi_similar_neigh.loc[((poi_similar_neigh['Venue Category'] == interested_cat[0]) & (poi_similar_neigh['Distance(km)']<2)) |  
                      ((poi_similar_neigh['Venue Category'] == interested_cat[1]) & (poi_similar_neigh['Distance(km)']<1)) | 
                      ((poi_similar_neigh['Venue Category'].isin(interested_cat[2:])) & (poi_similar_neigh['Distance(km)']<2)),
                      poi_similar_neigh.columns[[0]+[3]+[6]]]

#display summary table
summary.groupby(['District', 'Venue Category']).count()

Venue
District       Venue Category             
Bang Rak       Coffee Shop               1
               Gym / Fitness Center      1
Khlong San     Coffee Shop               2
Pathum Wan     Coffee Shop               1
               Gym / Fitness Center      2
               Supermarket               2
Ratchathewi    Convenience Store         1
               Gym / Fitness Center      1
               Supermarket               2
Samphanthawong Coffee Shop               2
Sathon         Gym / Fitness Center      2

# Results
The table above has shown that only _'Pathum Wan'_ district has all the required venues within the preferred distance. From the table, I can see that there are 1 Coffee Shop within 1km, 2 Gyms within 2km, and 2 Supermarkets within 2km. This has narrowed down my search for a new home, and it looks like I will be looking for property in _'Pathum Wan'_ district in Bangkok!